### Import and Load data

In [16]:
from numpy import loadtxt
import numpy as np
from xgboost import XGBClassifier
import pandas as pd
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

In [5]:
# one hot encoding
def onehot(y):
    label = []
    for i in y:
        label.append([int(i)])
    y_result = to_categorical(label).astype(int)
    
    return y_result

In [6]:
# load data
train = loadtxt('../data/csvs/train_data.csv', delimiter=",")
test = loadtxt('../data/csvs/test_data.csv', delimiter=",")

In [7]:
X_train = train[:, :-1]
y_train_split = train[:, -1]
X_test = test[:, :-1]
y_test_split = test[:, -1]

In [8]:
y_train = onehot(y_train_split)
y_test = onehot(y_test_split)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(139, 22)
(15, 22)
(139, 3)
(15, 3)


In [10]:
# fit model no training data
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state=32)
model.fit(X_test, y_test)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=32, ...)


In [11]:
expected_y = y_test
predicted_y = model.predict(X_test)

In [36]:
print(expected_y)
print(predicted_y)

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [15]:
predicted_y[0]

array([1., 0., 0.])

In [14]:
accuracy = accuracy_score(expected_y, predicted_y)

In [13]:
print("Accuracy: %.2f%%" % (accuracy * 100))

Accuracy: 100.00%


In [17]:
actions = np.array(['hands_up', 'neck_side', 'neck_down'])
actions[np.argmax(predicted_y[0])]

'hands_up'

### Save Weights

In [18]:
model.save_model("first_model.json")